In [41]:
import openai 
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
def invoke_llamafile(prompt):
    
    """
    This function interacts with the Llamafile model to get the response.
    
    Input:
    prompt: str
    
    Returns:
    response: str
    """
    client = openai.OpenAI(
        base_url="http://localhost:8080/v1",
        api_key="sk-no-key-required"
    )
    completion = client.chat.completions.create(
        model="LLaMA_CPP",
        messages=[
            {"role": "system", "content": "You are an experienced vitreoretinal surgeon speaking with trainees. You can answer detailed questions about vitreoretinal surgery concisely and accurately. Limit your responses to one sentence ONLY."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return completion.choices[0].message.content

In [53]:
def invoke_llama_only(prompt):
    
    """
    This function interacts with the ONLY the Llamafile model to get the response (no RAG prompting)
    
    Input:
    prompt: str
    
    Returns:
    response: str
    """
    client = openai.OpenAI(
        base_url="http://localhost:8080/v1",
        api_key="sk-no-key-required"
    )
    completion = client.chat.completions.create(
        model="LLaMA_CPP",
        messages=[
            {"role": "system", "content": ""},
            {"role": "user", "content": prompt}
        ]
    )
    
    return completion.choices[0].message.content

In [54]:
def query_rag(query_text: str):
    
    '''
    This function takes a query text and returns the response from the RAG model
    
    Input:
    
    query_text: str
    
    Returns:
    
    response_rag: str
    
    '''
    # Get the embedding function and search the database
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=chroma_path, embedding_function=embedding_function)

    # Search the database for the most similar chunks, and return the top 3
    results = db.similarity_search_with_score(query_text, k=3)

    # Prepare context text
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # Instantiate and use the Llamafile to get the response
    response_rag = invoke_llamafile(prompt)
    
    response_llm_only = invoke_llama_only(query_text)
  
  
    return response_rag , response_llm_only 

In [55]:
def calculate_cosine_similarity(text1, text2):
    '''
    
    This function calculates the cosine similarity between two text strings.
    
    Input:
    
    text1: str
    
    text2: str
    
    Returns:
    
    cosine_similarity: float
    
    '''
    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Vectorize the text strings
    tfidf_matrix = vectorizer.fit_transform([text1, text2])

    # Compute the cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    return cosine_sim[0][0]

Cosine Similarity: 1.0


In [68]:
query_text = "What are the most signficant non-modifiable risk factors for age related macular degeneration?"
ideal_response = "Nonmodifiable risk factors for age related macular degeneration include older age and genetic risk."

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_1 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_1 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_1}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_1}")

Ideal Response: Nonmodifiable risk factors for age related macular degeneration include older age and genetic risk.
Response from RAG:  Age and genetics are the most significant non-modifiable risk factors for age-related macular degeneration.</s>
Response from LLM only:  Age is the most significant non-modifiable risk factor for Age-Related Macular Degeneration (AMD). AMD is a progressive eye condition that mainly affects older adults, with the risk increasing significantly after the age of 50. Other non-modifiable risk factors include:

1. Genetics: Having a family history of AMD increases the risk of developing the condition. Certain genes have been linked to AMD, such as the Complement Factor H (CFH) and Complement Factor H Y402H (CFHY402H) variants.

2. Ethnicity: AMD is more common in Caucasians than in other ethnic groups. African Americans, Hispanics, and Asians are at a lower risk but still develop AMD as they age.

3. Gender: Women are more likely to develop AMD than men.

4.

In [67]:
query_text = "What is the role of antioxidant vitamins in age related macular degeneration?"
ideal_response = "There is lack of conclusive evidence about the benefit of multivitamin supplements in AMD, but antioxidant vitamins (A, C, and E), may help reduce the risk and progression of age-related macular degeneration by neutralizing free radicals"

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_2 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_2 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_2}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_2}")




Ideal Response: There is lack of conclusive evidence about the benefit of multivitamin supplements in AMD, but antioxidant vitamins (A, C, and E), may help reduce the risk and progression of age-related macular degeneration by neutralizing free radicals
Response from RAG:  Antioxidant vitamins, particularly A, C, and E, may help reduce the risk and progression of age-related macular degeneration by neutralizing free radicals and protecting the retina from oxidative damage. However, their role in preventing legal blindness is not definitively established. (Sources: 8,9)</s>
Response from LLM only:  Antioxidant vitamins, particularly A, C, and E, have been studied extensively in relation to Age-Related Macular Degeneration (AMD) due to their role in protecting the eyes from oxidative damage. Oxidative stress is believed to play a significant role in the development and progression of AMD.

The Age-Related Eye Disease Study (AREDS) and the Age-Related Eye Disease Study 2 (AREDS2) are two 

In [72]:
query_text = "What are some serious risks of anti-VEGF therapy for age related macular degeneration?"
ideal_response = "Serious risks of anti-VEGF therapy include infection and retinal detachment"

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_3 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_3 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_3}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_3}")

Ideal Response: Serious risks of anti-VEGF therapy include infection and retinal detachment
Response from RAG:  The serious risks of anti-VEGF therapy for age related macular degeneration include infection and retinal detachment. These risks should be interpreted in the context that patients receive multiple injections over long periods of time.</s>
Response from LLM only:  Anti-VEGF therapy is a common treatment for age-related macular degeneration (AMD), which is a leading cause of vision loss in older adults. The goal of this therapy is to inhibit the action of vascular endothelial growth factor (VEGF), a protein that promotes the growth of new blood vessels in the eye, which can lead to vision loss in AMD.

While anti-VEGF therapy has been shown to be effective in slowing the progression of AMD and improving vision in some patients, it does carry some risks. Some of the most serious risks include:

1. Intraocular Infections: Anti-VEGF therapy increases the risk of developing intrao

In [73]:
query_text = "Is the course of retinopathy of prematurity (ROP) more correlated with postmenstrual age or postnatal age?"
ideal_response = "The course of ROP is more correlated with postmenstrual age (PMA) than postnatal age"

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_4 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_4 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_4}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_4}")

Ideal Response: The course of ROP is more correlated with postmenstrual age (PMA) than postnatal age
Response from RAG:  Retinopathy of prematurity (ROP) progression is more correlated with postmenstrual age.</s>
Response from LLM only:  The course of Retinopathy of Prematurity (ROP) is more closely related to postmenstrual age rather than postnatal age. This is because ROP is primarily a disease of prematurity, and its development is influenced by the gestational age at birth. Postmenstrual age is a more accurate indicator of the developmental stage of the retina at the time of birth, which is crucial in understanding the risk and progression of ROP. Postnatal age, on the other hand, only reflects the time elapsed since birth and may not necessarily reflect the developmental stage of the retina.</s>
Similarity between LLM response and ideal response: 0.5197007620335637
Similarity between RAG response and ideal response: 0.527835809618309


In [74]:
query_text = "When do we initiate retinal examinations for infants born at 22 to 27 weeks and for infants born at 28 weeks or more?"
ideal_response = "We initiate retinal examinations at 31 weeks postmenstrual age (PMA) for infants born at 22 to 27 weeks and at four weeks of chronologic age for infants bornat ≥28 weeks"

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_5 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_5 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_5}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_5}")

Ideal Response: We initiate retinal examinations at 31 weeks postmenstrual age(PMA) for infants born at 22 to 27 weeks and at four weeks of chronologic age for infants bornat ≥28 weeks
Response from RAG:  Infants born at 22 to 27 weeks should undergo retinal examinations starting at 4 to 6 weeks of age, while those born at 28 weeks or more should have their first retinal examination between 6 to 12 weeks of age. (RP: retinopathy of prematurity)</s>
Response from LLM only:  The American Academy of Pediatrics (AAP) recommends that infants born at or before 27 weeks of gestation undergo a comprehensive ocular examination within the first 48 hours of life, and then at regular intervals based on the infant's clinical condition and risk factors for ocular diseases. This is due to the higher risk of ocular complications, such as retinopathy of prematurity (ROP), in premature infants.

For infants born at 28 weeks or more, the AAP recommends that they undergo a comprehensive ocular examination

In [77]:
query_text = "What is on the differential diagnosis of panuveitis?"
ideal_response = "The differential diagnosis of panuveitis includes bacterial and viral infections, scleritis, uveitis, and various systemic diseases such as Behçet's disease, Vogt-Koyanagi-Harada disease, and birdshot chorioretinopathy"

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_6 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_6 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_6}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_6}")

Ideal Response: The differential diagnosis of panuveitis includes bacterial and viral infections, scleritis, uveitis, and various systemic diseases such as Behçet's disease, Vogt-Koyanagi-Harada disease, and birdshot chorioretinopathy
Response from RAG:  The differential diagnosis of panuveitis includes bacterial and viral infections, scleritis, uveitis, and various systemic diseases such as Behçet's disease, Vogt-Koyanagi-Harada disease, and birdshot chorioretinopathy. Additionally, it can be associated with conditions like tuberculosis, syphilis, and sarcoidosis. In some cases, it may be secondary to trauma or intraocular surgery.</s>
Response from LLM only:  Panuveitis is a type of uveitis that affects the entire uveoscleral system, including the iris, ciliary body, choroid, retina, and vitreous. The differential diagnosis of panuveitis includes various inflammatory and infectious conditions. Here are some of the most common causes of panuveitis:

1. Infectious causes:
   * Viral in

In [81]:
query_text = "What is the role of intravitreal glucocorticoids in the therapy of retinal vein occlusion (RVO)."
ideal_response = "Intravitreal glucocorticoids can be used off-label to treat macular edema associated with retinal vein occlusion (RVO), improving visual outcomes."

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_7 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_7 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_7}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_7}")

Ideal Response: Intravitreal glucocorticoids can be used off-label to treat macular edema associated with retinal vein occlusion (RVO), improving visual outcomes.
Response from RAG:  Intravitreal glucocorticoids can be used off-label to treat macular edema associated with retinal vein occlusion (RVO), improving visual outcomes. However, long-term safety and efficacy have not been established.</s>
Response from LLM only:  Intravitreal glucocorticoids, such as triamcinolone acetonide and dexamethasone, have been used in the therapy of retinal vein occlusion (RVO) as an alternative or adjunctive treatment to anti-vascular endothelial growth factor (anti-VEGF) agents. The primary role of intravitreal glucocorticoids in RVO therapy is to reduce inflammation and retinal edema.

RVO is a common retinal vascular disease characterized by the obstruction of retinal veins, leading to retinal ischemia, edema, and neovascularization. The inflammatory response plays a crucial role in the pathogenesi

In [82]:
query_text = "What is the most common type of retinal detachment?"
ideal_response = "A rhegmatogenous retinal detachment (RRD) is themost common type of retinal detachment."

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_8 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_8 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_8}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_8}")

Ideal Response: A rhegmatogenous retinal detachment (RRD) is themost common type of retinal detachment.
Response from RAG:  Rhegmatogenous retinal detachment (RRD) is the most common type of retinal detachment.</s>
Response from LLM only:  The most common type of retinal detachment is called "rhegmatogenous retinal detachment." This occurs when there is a tear or hole in the retina, allowing fluid from the vitreous gel inside the eye to accumulate under the retina and cause it to detach. This type of retinal detachment accounts for about 90% of all cases. Other less common types of retinal detachment include traumatic retinal detachment (caused by injury to the eye), tractional retinal detachment (caused by scar tissue or other conditions that pull on the retina), and exudative retinal detachment (caused by fluid accumulation under the retina from conditions such as inflammation or cancer).</s>
Similarity between LLM response and ideal response: 0.42516841569703284
Similarity between R

In [85]:
query_text = "What are the primary challenges of anti-VEGF treatment?"
ideal_response = "A primary challenge of VEGF inhibitor therapy is the need for repeated intravitreal injections, which contributes to nonadherence and undertreatment"

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_9 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_9 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_9}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_9}")

Ideal Response: A primary challenge of VEGF inhibitor therapy is the need for repeated intravitreal injections, which contributes to nonadherence and undertreatment
Response from RAG:  The primary challenges of anti-VEGF treatment include the risk of infection and retinal detachment, the need for regular injections to maintain efficacy, and the potential for recurrence or progression of proliferative diabetic retinopathy with cessation of therapy. Long-term safety and efficacy have not been established.</s>
Response from LLM only:  Anti-vascular endothelial growth factor (anti-VEGF) treatment is a common therapeutic approach for various conditions, including age-related macular degeneration (AMD), diabetic retinopathy, and macular edema. While anti-VEGF therapy has revolutionized the management of these conditions, it also comes with several challenges:

1. Intravitreal injections: Anti-VEGF therapy is typically administered as intravitreal injections, which can be uncomfortable for pa

In [84]:
query_text = "What are the goals of diabetic retinopathy treatment?"
ideal_response = "The goals of treatment of diabetic retinopathy include improvement in vision, preservation of vision, and reduction in the rate of progression and frequency of retinopathy, vitreoushemorrhage, and macular edema."

# ask the llm to generate response for the query text

response_rag, response_llm_only = query_rag(query_text)

print(f"Ideal Response: {ideal_response}")

print(f"Response from RAG: {response_rag}")

print(f"Response from LLM only: {response_llm_only}")

similarity_llm_ideal_10 = calculate_cosine_similarity(response_llm_only, ideal_response)
similarity_rag_ideal_10 = calculate_cosine_similarity(response_rag, ideal_response)

print(f"Similarity between LLM response and ideal response: {similarity_llm_ideal_10}")  
print(f"Similarity between RAG response and ideal response: {similarity_rag_ideal_10}")

Ideal Response: The goals of treatment of diabetic retinopathy include improvement in vision, preservation of vision, and reduction in the rate of progression and frequency of retinopathy, vitreoushemorrhage, and macular edema.
Response from RAG:  The goals of diabetic retinopathy treatment are to prevent progression of proliferative diabetic retinopathy and subsequent visual loss, with effective modalities including ranibizumab and panretinal photocoagulation. Regular anti-VEGF treatment may improve proliferative changes and minimize the necessity of panretinal photocoagulation, but there is a risk of recurrence and progression with cessation of therapy. Long-term efficacy and safety of anti-VEGF therapy have not been established.</s>
Response from LLM only:  The primary goals of diabetic retinopathy (DR) treatment are:

1. Prevent or slow down the progression of DR: The main objective of DR treatment is to prevent or slow down the worsening of the condition. This can be achieved thro